# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [16]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [17]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head(5)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Meulaboh,ID,74.10,overcast clouds,4.1363,96.1285,Hotel Meuligou
1,2,Atar,MR,76.57,clear sky,20.5169,-13.0499,فندق سكليل
2,8,Gold Coast,AU,83.16,overcast clouds,-28.0000,153.4333,Zenith
3,9,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,Pono Kai Resort
4,11,Sotouboua,TG,75.61,clear sky,8.5667,0.9833,Verger


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [18]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Max Temp,Hotel Name,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [25]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City_ID"] == 305]
vacation_end = vacation_df.loc[vacation_df["City_ID"] == 305]
vacation_stop1 = vacation_df.loc[vacation_df["City_ID"] == 346]
vacation_stop2 = vacation_df.loc[vacation_df["City_ID"] == 578]
vacation_stop3 = vacation_df.loc[vacation_df["City_ID"] == 600]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [30]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2,
                                         vacation_stop3, vacation_end])

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
73,305,Acapulco,MX,78.15,scattered clouds,16.8634,-99.8901,Hotel del Valle
82,346,Isla Mujeres,MX,75.25,clear sky,21.2311,-86.7310,Villa La Bella
127,578,Carmen,MX,78.84,few clouds,18.6333,-91.8333,Holiday Inn Express
133,600,Lazaro Cardenas,MX,77.79,clear sky,17.9583,-102.2000,Hotel Sol del Pacífico
73,305,Acapulco,MX,78.15,scattered clouds,16.8634,-99.8901,Hotel del Valle


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [58]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lng", "Lat"]].copy()

# Display sample data
waypoints_df

,Lng,Lat
73,-99.8901,16.8634
82,-86.7310,21.2311
127,-91.8333,18.6333
133,-102.2000,17.9583
73,-99.8901,16.8634


## Use GeoViews to create map that shows the four cities in the itinerary

In [59]:
# Configure the map plot by using the itineraty_df
waypoints_map = waypoints_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "purple")

In [60]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [61]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [62]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '48bfb7c90c424406b8af14821aab4a04',
 'waypoints': '16.8634,-99.8901|21.2311,-86.731|18.6333,-91.8333|17.9583,-102.2|16.8634,-99.8901'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [100]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-99.8901, 16.8634], 'original_index': 0},
     {'location': [-86.731, 21.2311], 'original_index': 1},
     {'location': [-91.8333, 18.6333], 'original_index': 2},
     {'location': [-102.2, 17.9583], 'original_index': 3},
     {'location': [-99.8901, 16.8634], 'original_index': 4}],
    'units': 'metric',
    'distance': 4491312,
    'distance_units': 'meters',
    'time': 167823.755,
    'legs': [{'distance': 1915353,
      'time': 71542.234,
      'steps': [{'from_index': 0,
        'to_index': 1,
        'distance': 30,
        'time': 3.676,
        'instruction': {'text': 'Drive south on Calle Vicente Guerrero.'}},
       {'from_index': 1,
        'to_index': 3,
        'distance': 44,
        'time': 11.016,
        'instruction': {'text': 'Turn left onto Avenida Sonora.'}},
       {'from_index': 3,
        'to_index': 5,
        'distance': 208,
        'time': 27.305,
        'i

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [204]:
# Fetch the route's legs coordinates from the JSON reponse
import json
legs = route_response["features"][0]['geometry']["coordinates"]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [242]:
for x in legs:
    for y in x:
        print(y)

[-99.890005, 16.863433]
[-99.889908, 16.863173]
[-99.889821, 16.863208]
[-99.889529, 16.863326]
[-99.888865, 16.861719]
[-99.888795, 16.861596]
[-99.886922, 16.862524]
[-99.886252, 16.862814]
[-99.885929, 16.862932]
[-99.885241, 16.863157]
[-99.884813, 16.863321]
[-99.884299, 16.863477]
[-99.883771, 16.863641]
[-99.882228, 16.864081]
[-99.878291, 16.865203]
[-99.878176, 16.865237]
[-99.877718, 16.865377]
[-99.874925, 16.866225]
[-99.87454, 16.866349]
[-99.87436, 16.866424]
[-99.874119, 16.866558]
[-99.873876, 16.86671]
[-99.873623, 16.866917]
[-99.873231, 16.86724]
[-99.872935, 16.867548]
[-99.872765, 16.867761]
[-99.87265, 16.867969]
[-99.872544, 16.868213]
[-99.872501, 16.868415]
[-99.872461, 16.868711]
[-99.872373, 16.870575]
[-99.872359, 16.870741]
[-99.87227, 16.870843]
[-99.872043, 16.871056]
[-99.871888, 16.871172]
[-99.871716, 16.871264]
[-99.871537, 16.871337]
[-99.871359, 16.871384]
[-99.871166, 16.871416]
[-99.869948, 16.871602]
[-99.868809, 16.871774]
[-99.86847, 16.871843]

[-99.508839, 17.55282]
[-99.508922, 17.553378]
[-99.509116, 17.554532]
[-99.509254, 17.555315]
[-99.509464, 17.556505]
[-99.509735, 17.558121]
[-99.509861, 17.558744]
[-99.510085, 17.560177]
[-99.510248, 17.561094]
[-99.510308, 17.561469]
[-99.510424, 17.562144]
[-99.510485, 17.56252]
[-99.510682, 17.563518]
[-99.510895, 17.564773]
[-99.51105, 17.565643]
[-99.511288, 17.56704]
[-99.511356, 17.567426]
[-99.511388, 17.567596]
[-99.511465, 17.568]
[-99.511661, 17.568878]
[-99.511838, 17.569554]
[-99.511955, 17.569969]
[-99.512235, 17.570956]
[-99.512438, 17.571643]
[-99.512525, 17.572004]
[-99.512571, 17.572202]
[-99.51278, 17.573024]
[-99.513001, 17.57385]
[-99.51314, 17.574517]
[-99.513164, 17.574609]
[-99.513311, 17.575236]
[-99.5136, 17.576384]
[-99.513685, 17.576669]
[-99.5137, 17.576752]
[-99.513708, 17.576793]
[-99.513892, 17.577451]
[-99.514088, 17.578066]
[-99.51412, 17.578181]
[-99.514313, 17.578848]
[-99.514431, 17.579386]
[-99.514468, 17.579531]
[-99.514617, 17.580186]
[-99.51

[-99.205252, 18.108356]
[-99.205125, 18.10864]
[-99.20502, 18.10901]
[-99.204882, 18.109467]
[-99.204767, 18.109964]
[-99.204628, 18.110895]
[-99.204608, 18.111108]
[-99.204582, 18.111506]
[-99.204569, 18.112031]
[-99.204612, 18.112718]
[-99.204671, 18.113252]
[-99.204755, 18.113865]
[-99.205005, 18.114901]
[-99.205209, 18.115483]
[-99.205387, 18.115992]
[-99.205603, 18.116607]
[-99.205905, 18.117454]
[-99.206244, 18.118489]
[-99.20647, 18.119133]
[-99.206648, 18.119628]
[-99.206967, 18.120571]
[-99.207283, 18.12152]
[-99.208215, 18.124279]
[-99.208366, 18.124797]
[-99.20851, 18.125416]
[-99.208603, 18.126092]
[-99.208646, 18.126649]
[-99.208652, 18.12727]
[-99.208607, 18.128503]
[-99.208514, 18.130162]
[-99.208415, 18.131954]
[-99.208312, 18.133597]
[-99.208218, 18.13513]
[-99.20821, 18.135605]
[-99.208239, 18.136076]
[-99.208312, 18.136721]
[-99.208412, 18.137287]
[-99.208545, 18.137853]
[-99.208704, 18.138337]
[-99.208841, 18.138716]
[-99.209436, 18.140109]
[-99.209962, 18.141396]
[

[-98.393507, 18.939833]
[-98.392672, 18.93916]
[-98.391817, 18.938478]
[-98.39117, 18.937988]
[-98.390878, 18.937774]
[-98.390863, 18.937742]
[-98.390843, 18.937713]
[-98.390819, 18.937685]
[-98.390791, 18.93766]
[-98.390431, 18.937394]
[-98.390383, 18.93735]
[-98.390339, 18.937302]
[-98.390301, 18.93725]
[-98.390273, 18.937203]
[-98.390249, 18.937154]
[-98.390229, 18.937103]
[-98.390208, 18.937031]
[-98.390196, 18.936958]
[-98.390194, 18.936883]
[-98.390202, 18.936809]
[-98.390219, 18.936737]
[-98.390246, 18.936667]
[-98.390282, 18.9366]
[-98.390327, 18.936539]
[-98.390379, 18.936483]
[-98.390439, 18.936434]
[-98.390504, 18.936393]
[-98.390574, 18.93636]
[-98.390649, 18.936335]
[-98.390725, 18.936319]
[-98.390804, 18.936312]
[-98.390883, 18.936315]
[-98.39096, 18.936327]
[-98.391036, 18.936348]
[-98.391108, 18.936378]
[-98.391175, 18.936416]
[-98.391237, 18.936462]
[-98.391293, 18.936515]
[-98.391341, 18.936574]
[-98.39138, 18.936639]
[-98.391411, 18.936707]
[-98.391433, 18.936779]
[-

[-97.467349, 18.828674]
[-97.467093, 18.828815]
[-97.466899, 18.82897]
[-97.466688, 18.82919]
[-97.466514, 18.829404]
[-97.466338, 18.829664]
[-97.466235, 18.829858]
[-97.466131, 18.830164]
[-97.465914, 18.831118]
[-97.465841, 18.831404]
[-97.465778, 18.831607]
[-97.465696, 18.831787]
[-97.465583, 18.83197]
[-97.465302, 18.83232]
[-97.46509, 18.83253]
[-97.464775, 18.832767]
[-97.463821, 18.833242]
[-97.46321, 18.833617]
[-97.462697, 18.834103]
[-97.462498, 18.834372]
[-97.462351, 18.834642]
[-97.461827, 18.83608]
[-97.461737, 18.83629]
[-97.461648, 18.836446]
[-97.461519, 18.836639]
[-97.461411, 18.836767]
[-97.461201, 18.836985]
[-97.460988, 18.837165]
[-97.460743, 18.837313]
[-97.460503, 18.837422]
[-97.460271, 18.837492]
[-97.460066, 18.837532]
[-97.459771, 18.837559]
[-97.459197, 18.837527]
[-97.457776, 18.837105]
[-97.457328, 18.837033]
[-97.456873, 18.836998]
[-97.456103, 18.837066]
[-97.455525, 18.837241]
[-97.455122, 18.837417]
[-97.425213, 18.850991]
[-97.42187, 18.852605]
[-

...

[-101.406869, 19.632812]
[-101.407133, 19.632414]
[-101.407502, 19.631893]
[-101.407883, 19.631395]
[-101.408307, 19.63088]
[-101.408655, 19.630469]
[-101.409113, 19.629965]
[-101.409511, 19.629554]
[-101.409848, 19.629216]
[-101.410155, 19.628933]
[-101.410599, 19.628523]
[-101.41099, 19.628155]
[-101.411423, 19.627743]
[-101.413481, 19.625838]
[-101.413876, 19.625464]
[-101.41432, 19.625053]
[-101.414786, 19.624617]
[-101.415286, 19.624141]
[-101.418962, 19.620712]
[-101.419327, 19.620379]
[-101.419706, 19.620035]
[-101.420165, 19.619655]
[-101.42043, 19.619441]
[-101.420709, 19.619236]
[-101.421215, 19.618886]
[-101.421513, 19.618685]
[-101.421797, 19.618504]
[-101.422274, 19.618236]
[-101.422703, 19.618]
[-101.42319, 19.617752]
[-101.423621, 19.617548]
[-101.423923, 19.617427]
[-101.424156, 19.617336]
[-101.426746, 19.616345]
[-101.427384, 19.616088]
[-101.428533, 19.61563]
[-101.429644, 19.615177]
[-101.429842, 19.615102]
[-101.430028, 19.615026]
[-101.430396, 19.614864]
[-101.430

[-102.060039, 18.821524]
[-102.059814, 18.82093]
[-102.059759, 18.820795]
[-102.059304, 18.819871]
[-102.058661, 18.81832]
[-102.05847, 18.817874]
[-102.058442, 18.817814]
[-102.058194, 18.817372]
[-102.057895, 18.816789]
[-102.057633, 18.816239]
[-102.057554, 18.81604]
[-102.057502, 18.81589]
[-102.057428, 18.815624]
[-102.05739, 18.815391]
[-102.057384, 18.815339]
[-102.057368, 18.815031]
[-102.057368, 18.814924]
[-102.057436, 18.814312]
[-102.058219, 18.810687]
[-102.058372, 18.810004]
[-102.058461, 18.809691]
[-102.058653, 18.809165]
[-102.058767, 18.808946]
[-102.058878, 18.808756]
[-102.059045, 18.808498]
[-102.059355, 18.808137]
[-102.059677, 18.807795]
[-102.060416, 18.807031]
[-102.063927, 18.803387]
[-102.064187, 18.803103]
[-102.064446, 18.80276]
[-102.06462, 18.802465]
[-102.064732, 18.802247]
[-102.064895, 18.801837]
[-102.064934, 18.801686]
[-102.064996, 18.801367]
[-102.065032, 18.801084]
[-102.065042, 18.80068]
[-102.065016, 18.800423]
[-102.064985, 18.800185]
[-102.064

[-101.909365, 18.107745]
[-101.910231, 18.106478]
[-101.910684, 18.105811]
[-101.911928, 18.104093]
[-101.912645, 18.103108]
[-101.913375, 18.102101]
[-101.914166, 18.100997]
[-101.914941, 18.099918]
[-101.915415, 18.099178]
[-101.916034, 18.098118]
[-101.916129, 18.097956]
[-101.916357, 18.097547]
[-101.916537, 18.097175]
[-101.916701, 18.096803]
[-101.917008, 18.096115]
[-101.917151, 18.095776]
[-101.917301, 18.095426]
[-101.917586, 18.094701]
[-101.91782, 18.094117]
[-101.918056, 18.093542]
[-101.919022, 18.091209]
[-101.919976, 18.088861]
[-101.920391, 18.087826]
[-101.92083, 18.086763]
[-101.921072, 18.08618]
[-101.921513, 18.085032]
[-101.922886, 18.081633]
[-101.923254, 18.080771]
[-101.923485, 18.080291]
[-101.923789, 18.079658]
[-101.924275, 18.078693]
[-101.924722, 18.077845]
[-101.926145, 18.07545]
[-101.926939, 18.074121]
[-101.927303, 18.073547]
[-101.927655, 18.072986]
[-101.928143, 18.072147]
[-101.928752, 18.071107]
[-101.929142, 18.070494]
[-101.929534, 18.069858]
[-10

[-101.303955, 17.551262]
[-101.303058, 17.55186]
[-101.302138, 17.552516]
[-101.302031, 17.552593]
[-101.301661, 17.552813]
[-101.301428, 17.55297]
[-101.30011, 17.553783]
[-101.299768, 17.553945]
[-101.299487, 17.554057]
[-101.299065, 17.554123]
[-101.298434, 17.554108]
[-101.29796, 17.554009]
[-101.297516, 17.553833]
[-101.29161, 17.551307]
[-101.291173, 17.551101]
[-101.29018, 17.550673]
[-101.289105, 17.550236]
[-101.288742, 17.550076]
[-101.288576, 17.550008]
[-101.288485, 17.549968]
[-101.287512, 17.549535]
[-101.287395, 17.549482]
[-101.287336, 17.549462]
[-101.286164, 17.548985]
[-101.285728, 17.548806]
[-101.285251, 17.548598]
[-101.283283, 17.547714]
[-101.283172, 17.547667]
[-101.281637, 17.547016]
[-101.28078, 17.546652]
[-101.280726, 17.546621]
[-101.280076, 17.54634]
[-101.2799, 17.546274]
[-101.279477, 17.546087]
[-101.279042, 17.545892]
[-101.278643, 17.54573]
[-101.278067, 17.54548]
[-101.277927, 17.545415]
[-101.277831, 17.545357]
[-101.277652, 17.545257]
[-101.277228

[-100.037778, 16.980555]
[-100.037521, 16.980499]
[-100.037317, 16.980396]
[-100.037092, 16.980278]
[-100.036904, 16.980121]
[-100.036743, 16.979962]
[-100.035726, 16.978455]
[-100.035085, 16.977505]
[-100.03427, 16.976309]
[-100.034082, 16.976098]
[-100.033927, 16.975985]
[-100.033777, 16.975896]
[-100.033523, 16.975781]
[-100.032748, 16.975581]
[-100.030166, 16.974945]
[-100.030047, 16.974916]
[-100.028733, 16.974597]
[-100.028466, 16.974532]
[-100.027618, 16.974303]
[-100.027273, 16.974216]
[-100.026411, 16.974001]
[-100.025289, 16.973684]
[-100.023838, 16.973312]
[-100.023668, 16.973265]
[-100.023376, 16.973191]
[-100.020732, 16.972526]
[-100.020394, 16.972445]
[-100.01942, 16.972195]
[-100.018042, 16.971843]
[-100.017847, 16.971793]
[-100.017756, 16.971772]
[-100.017175, 16.971643]
[-100.016794, 16.97155]
[-100.01571, 16.9713]
[-100.0142, 16.970914]
[-100.013993, 16.970856]
[-100.013808, 16.970788]
[-100.013614, 16.9707]
[-100.013405, 16.970588]
[-100.012286, 16.969773]
[-100.0119

In [251]:
legs[2]

[[-91.833092, 18.633418],
 [-91.832975, 18.633233],
 [-91.832629, 18.632712],
 [-91.832401, 18.632435],
 [-91.832199, 18.632493],
 [-91.831973, 18.632565],
 [-91.831564, 18.632688],
 [-91.831081, 18.632841],
 [-91.830491, 18.633032],
 [-91.829847, 18.631763],
 [-91.829798, 18.631707],
 [-91.829747, 18.631668],
 [-91.829586, 18.631539],
 [-91.829359, 18.63105],
 [-91.829023, 18.630852],
 [-91.828502, 18.630684],
 [-91.828052, 18.63057],
 [-91.827692, 18.630475],
 [-91.827446, 18.630449],
 [-91.82719, 18.630444],
 [-91.826873, 18.630444],
 [-91.826655, 18.630372],
 [-91.826586, 18.630358],
 [-91.826518, 18.630341],
 [-91.82622, 18.630341],
 [-91.825843, 18.6304],
 [-91.82582, 18.630342],
 [-91.82577, 18.630185],
 [-91.825793, 18.630175],
 [-91.857491, 18.612106],
 [-91.857749, 18.612043],
 [-91.857896, 18.612038],
 [-91.858012, 18.612064],
 [-91.858182, 18.61211],
 [-91.859124, 18.612428],
 [-91.859589, 18.612546],
 [-91.860769, 18.612846],
 [-91.862543, 18.61331],
 [-91.876167, 18.61687

In [210]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
# YOUR CODE HERE

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [14]:
# Create an empty DataFrame to store the steps' coordinates
route_df = # YOUR CODE HERE

# Add the steps' longitude and latitude from each step as columns to the DataFrame
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,longitude,latitude
0,12.479944,41.889982
1,12.479927,41.890012
2,12.479760,41.890249
3,12.479580,41.890493
4,12.479418,41.890662


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [15]:
# Configure the route path by using the GeoViews' Path function
route_path = # YOUR CODE HERE

In [16]:
# Display a composed plot by using the route_map and route_path objects
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)
   .Path.I   :Path   [longitude,latitude]